In [1]:
from Paciente import Paciente

In [2]:
import numpy as np
from scipy.ndimage import label, center_of_mass, sum  
from tqdm import tqdm

In [3]:
# Se instancia la clase para el paciente numero 29
paciente29 = Paciente(29, 3)

No damaged files


Construyendo Mascaras: 100%|██████████| 348/348 [00:18<00:00, 19.27it/s]


In [7]:
pat_height = paciente29.altura
pat_weight = paciente29.peso
spacing = paciente29.spacing
data = paciente29.mascaraGeneral

pat_BSA = np.sqrt((pat_height*pat_weight)/3600) #m^2? 

# Separate the different lesions and give them different labels
# Not connected regions will be seen as different objects 
labeled_data, num_features = label(data)
seg = np.argwhere(labeled_data)
print('Number of lesions detected: ' + str(num_features))

Number of lesions detected: 19


In [1]:
def weighted_distance(p1, p2, weights, type_of_distance):
    # Implementa aquí la lógica para calcular la distancia ponderada

    q = p1 - p2

    if type_of_distance == 'euclidian':
        # Weighted Euclidian distance
        return np.sqrt(((weights * q)**2).sum())

    elif type_of_distance == 'manhattan' or type_of_distance == 'cityblock':
        # Weighted Manhattan or Cityblock distance 
        return (weights * abs(q)).sum()

    elif type_of_distance == 'chebyshev':
        # Weighted Chebyshev distance 
        return max(weights * abs(q))

    else: 
        # Si se proporciona una cadena diferente a los tipos de distancia posibles, 
        # se imprime el siguiente mensaje de error. 
        print("Error: The selected distance type is not available. \
                Try 'euclidian', 'manhattan' or 'cityblock', or 'chebyshev'.")
        return None

def distances(segmentation, weights):
    # Implementa aquí el cálculo de las distancias entre pares de voxels
    euc_dist = []
    man_dist = []
    che_dist = []

    # Calcula el total de iteraciones necesarias para el progreso
    total_iterations = len(segmentation) * (len(segmentation) - 1)

    # Crea una barra de progreso con tqdm
    with tqdm(total=total_iterations) as pbar:
        for i in range(len(segmentation)):
            for j in range(len(segmentation)):
                if i != j:
                    euc_dist.append(weighted_distance(segmentation[i], segmentation[j], weights, 'euclidian'))
                    man_dist.append(weighted_distance(segmentation[i], segmentation[j], weights, 'manhattan'))
                    che_dist.append(weighted_distance(segmentation[i], segmentation[j], weights, 'chebyshev'))
                    # Actualiza la barra de progreso después de cada iteración
                    pbar.update(1)

    return euc_dist, man_dist, che_dist


In [ ]:
# Aplica las funciones a tus datos
euc_dist_vox, man_dist_vox, che_dist_vox = distances(seg, spacing)

In [ ]:
# Function that gives back all the euclidian distances between one specific 
# lesion and all the others 
def lesion_distances(index): 
    # Give back the array of euclidian distances between one specific lesion and the others
    d = dist_ctr[0, np.where(dist_ctr[3] == index)[0]]
    return d

In [ ]:
#%% Calculate parameters

# Calculation of parameters, based on the voxel distances
# Dmax: Max. distance between the two voxels that are the farthest away.
Dmax = max(euc_dist_vox)

# SDmax_euc_vox: Max. euclidian distance, normalized by BSA.
SDmax_euc_vox = Dmax/pat_BSA

# SDmax_man_vox: Max. Manhattan distance, normalized by BSA.
SDmax_man_vox = max(man_dist_vox)/pat_BSA

# SDmax_che_vox: Max. Chebyshev distance, normalized by BSA.
SDmax_che_vox = max(che_dist_vox)/pat_BSA


# Calculation of parameters, based on centroid distances 
# Dmax_patient: (Euclidian) Distance between the two lesions that are the farthest away from each other. 
Dmax_patient = max(euc_dist_ctr)

# Dmax_bulk: (Euclidian) Distance between the largest lesion and the one the farthest away from it. 
# Find largest lesion first 
sizes_of_lesions = [sum(data, labeled_data, index = i)
                    for i in range(1, num_features+1)]
largest_size = max(sizes_of_lesions) # how many pixels does the largest lesion have?
largest_index = np.where(sizes_of_lesions == largest_size)[0][0] # which Python Index does the largest lesion have?
largest_label = largest_index + 1 # which Label does the largest lesion have?  
#print('The largest lesion has label index ' + str(largest_label))
# Calculate the parameter
Dmax_bulk = max(lesion_distances(largest_index))
   
# SPREAD_bulk: Sum of the euclidian distances between the largest lesion and all the other lesions.
Spread_bulk = np.sum(lesion_distances(largest_index))

# SPREAD_patient: Over all lesions, the maximum of the sum of distances from one lesion to all the others.
sums = [np.sum(lesion_distances(i)) for i in range(num_features)]
Spread_patient = max(sums)
  
# SDmax_euc: Dmax_patient, normalized by BSA
SDmax_euc = Dmax_patient/pat_BSA

# SDmax_man: Maximum Manhattan distance between lesions, normalized by BSA. 
SDmax_man = max(man_dist_ctr)/pat_BSA

# SDmax_che: Maximum Chebyshev distance between lesions, normalized by BSA. 
SDmax_che = max(che_dist_ctr)/pat_BSA

#%% Add important features to Results file (output)
pat_features = pat_features.assign(Height = [pat_height],
                                   Weight = [pat_weight], 
                                   BSA = [pat_BSA],
                                   Dmax = [Dmax], 
                                   SDmax_euc_vox = [SDmax_euc_vox],
                                   SDmax_man_vox = [SDmax_man_vox], 
                                   SDmax_che_vox = [SDmax_che_vox], 
                                   Dmax_patient = [Dmax_patient], 
                                   Dmax_bulk = [Dmax_bulk], 
                                   Spread_bulk = [Spread_bulk],
                                   Spread_patient = [Spread_patient],
                                   SDmax_euc =[SDmax_euc],
                                   SDmax_man = [SDmax_man],
                                   SDmax_che = [SDmax_che])

pat_features.to_excel(results_dir + '/Results_Patient.xlsx')

t1 = time.time()
total = t1-t0
print('Patient ' + pat + ' successfully added.')
print('Total time calculating distances for patient ' + pat + ': ' + str(total) + 's')